<a href="https://colab.research.google.com/github/abhinivesh-s/Semi-Supervised-Text/blob/main/MLP_with_GloVE_on_all_DataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB


1.   MLP



In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import torch.utils.data as Data
import pickle
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install numpy==1.16.2
import numpy as np

     |████████████████████████████████| 17.3MB 1.5MB/s 
ERROR: xarray 0.18.2 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.16.2 which is incompatible.
ERROR: pyerfa 2.0.0 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: pyarrow 3.0.0 has requirement numpy>=1.16.6, but you'll have numpy 1.16.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement numpy>=1.18.5, but you'll have numpy 1.16.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: cupy-cuda101 9.1.0 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: astropy 4.2.1 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incom

In [4]:
!pip install --upgrade scikit-learn  # Do this to use sklearn SelfTrainingClassifier

     |████████████████████████████████| 22.3MB 76.3MB/s 
ERROR: kapre 0.3.5 has requirement numpy>=1.18.5, but you'll have numpy 1.16.2 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
print(np.__version__)
import sklearn
print(sklearn.__version__)

1.16.2
0.24.2


In [3]:
max_words = 30000

In [4]:
%cd "/content/drive/My Drive/Colab Notebooks/MixText-master"
!ls

/content/drive/My Drive/Colab Notebooks/MixText-master
 code		     README.md
 data		     rf_imdb.xlsx
 data_temp	     rf_ng.xlsx
 enron_mlp.xlsx     'Running MixText v3.ipynb'
 enron_sgd.xlsx     'Running MixText v5 ISIS Data.ipynb'
 glove.6B.100d.txt  'Semi-Supervised sklearn v2.ipynb'
 glove.6B.200d.txt   sgd_imdb.xlsx
 glove.6B.300d.txt   sgd_ng.xlsx
 glove.6B.50d.txt    tf-mlp_imdb_GloVE.xlsx
 glove.6B.zip	     tf-mlp_imdb.xlsx
 LICENSE	     tf-mlp_ng_GloVE.xlsx
 lstm_imdb.xlsx      tf-mlp_ng.xlsx
 nb_imdb.xlsx	     word2vec_model.bin
 nb_ng.xlsx


In [5]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')
def train_val_split(labels, n_labeled_per_class, unlabeled_per_class, n_labels, seed=0):
    """Split the original training set into labeled training set, unlabeled training set, development set

    Arguments:
        labels {list} -- List of labeles for original training set
        n_labeled_per_class {int} -- Number of labeled data per class
        unlabeled_per_class {int} -- Number of unlabeled data per class
        n_labels {int} -- The number of classes

    Keyword Arguments:
        seed {int} -- [random seed of np.shuffle] (default: {0})

    Returns:
        [list] -- idx for labeled training set, unlabeled training set, development set
    """
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        if n_labels == 2:
            # IMDB
            
            
            
            n_unlabeled_per_class = unlabeled_per_class   #10, 100, 500, 1000, 2500
            train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            train_unlabeled_idxs.extend(idxs[500: 500 + n_unlabeled_per_class])
            val_idxs.extend(idxs[-2000:])
        
        
        
            # train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            # train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            # train_unlabeled_idxs.extend(
            #     idxs[500: 500 + 5000])
            # val_idxs.extend(idxs[-2000:])
            

    return train_labeled_idxs, train_unlabeled_idxs, val_idxs



import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data



def clean(train_df):
    
    train_df['review']=train_df['review'].apply(lambda z: remove_punctuations(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_html(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_url(z))
    # train_df['review']=train_df['review'].apply(lambda z: remove_emoji(z))
    
    train_df['review']=train_df['review'].apply(lambda z: word_tokenize(z))
    
    # lemmatizer = WordNetLemmatizer()
    # train_df['review']=train_df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    train_df['review']=train_df['review'].apply(lambda x: ' '.join(x))
    
    return train_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
#------------------For GloVE---------------------#
embedding_dim = 300
maxlen = 500


def get_embed_matrix(word_index):

    hits = 0
    misses = 0

    hit_words = []
    missed_words = []
    # Prepare embedding matrix
    

    embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (max_words, 100)
    for word, i in word_index.items():
        if i < max_words:
            embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros
                hits += 1
                hit_words.append(word)
            else:
                misses += 1
                missed_words.append(word)  


                
    print("Converted %d words (%d misses)" % (hits, misses))

    print("Missed words: ", missed_words)

    return embedding_matrix


    #------------------For GloVE---------------------#

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier

tf.random.set_seed(0)
def create_model(word_index):

    # int_sequences_input = keras.Input(shape=(None,), dtype="int64")
    # embedded_sequences = embedding_layer(int_sequences_input)
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_words, embedding_dim, input_length = maxlen),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(50),
    ])

    embedding_matrix = get_embed_matrix(word_index)
    model.layers[0].set_weights([embedding_matrix])
    model.layers[0].trainable = False

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  # loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [15]:
import os

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report



# Parameters
sdg_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(**sdg_params)),
])

# # Supervised Pipeline
# pipeline = Pipeline([
#     ('vect', CountVectorizer(**vectorizer_params)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', SVC(probability=True, gamma="auto")),
# ])

# SelfTraining Pipeline
st_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), verbose=True)),
])

# # SelfTraining Pipeline
# st_pipeline = Pipeline([
#     ('vect', CountVectorizer(**vectorizer_params)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', SelfTrainingClassifier(SVC(probability=True, gamma="auto") )),
# ])





# LabelSpreading Pipeline
ls_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    # LabelSpreading does not support dense matrices
    ('todense', FunctionTransformer(lambda x: x.todense())),
    ('clf', LabelSpreading()),
])


def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # if -1 in y_train:
    #   print("Y-PRED-PROBA", clf.predict_proba(X_train))

    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("Classification Report: \n", classification_report(y_test, y_pred))
    print("-" * 10)
    print()


from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow.keras.layers import Embedding

def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}

    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


       
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    


    # print('y_train:', y_train)
    with tf.device('/device:GPU:0'):
      clf.fit(X_train, y_train)
      y_pred = np.array(clf.predict(X_test))
      
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)
    dict1['roc-auc'] = roc_auc_score(y_test, y_pred)
    dict1['precision'] = precision_score(y_test, y_pred)
    dict1['recall'] = recall_score(y_test, y_pred)
    dict1['f1'] = f1_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1

In [16]:
%%time
path_to_glove_file =  "glove.6B."+str(embedding_dim)+"d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.
CPU times: user 16.3 s, sys: 715 ms, total: 17 s
Wall time: 16.9 s


In [17]:
# MLP for IMDB
# Parameters
# mnb_params = 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing. sequence import pad_sequences


mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=50,activation = 'relu',solver='adam',random_state=1)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8, max_features=30000)


if __name__ == "__main__":

    df_mlp_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      tf.keras.backend.clear_session()

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])




      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels  

      tokenizer = Tokenizer(num_words=max_words)
      tokenizer.fit_on_texts(np.concatenate((X_train, X_test)))
      sequences = tokenizer.texts_to_sequences(np.concatenate((X_train, X_test)))
      word_index = tokenizer.word_index                   
      print("Found %s unique tokens." % len(word_index)) 

      data = pad_sequences(sequences, maxlen=maxlen)

      X_train, X_test = data[:len(X_train)], data[len(X_train):]

      # print("Shape of X_test: ", len(X_test)) 




      # print("Y-TYPE: ", type(labels[0]))  
      


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = X_train[:2*n], labels[:2*n]
      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      model = KerasClassifier(build_fn=lambda: create_model(word_index), epochs=20, verbose=0)
      # Supervised Pipeline
      pipeline = Pipeline([
          # ('vect', CountVectorizer(**vectorizer_params)),
          # ('tfidf', TfidfTransformer()), 
          # # ('kbest', MySelectKBest(f_classif, k=20000)),         
          # ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
          ('clf', model)
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
        tf.keras.backend.clear_session()
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        model = KerasClassifier(build_fn=lambda: create_model(word_index), epochs=20, verbose=0)
        st_pipeline = Pipeline([
            # ('vect', CountVectorizer(**vectorizer_params)),
            # ('tfidf', TfidfTransformer()),
            # # ('kbest', MySelectKBest(f_classif, k=20000)),
            # ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
            ('clf', SelfTrainingClassifier(model, criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_mlp_imdb_2k = df_mlp_imdb_2k.append(temp, ignore_index=True)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Found 91511 unique tokens.
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Converted 26818 words (3181 misses)
Missed words:  ['itbr', 'moviebr', 'filmbr', 'youve', 'werent', 'timebr', 'shouldnt', 'thembr', 'hadnt', 'lowbudget', 'himbr', 'theyve', 'onebr', 'thatbr', 'thisbr', 'mebr', 'storybr', 'bmovie', 'herebr', 'wellbr', 'allbr', 'goodbr', 'couldve', 'overthetop', 'onbr', 'wouldve', 'endbr', 'isbr', 'waybr', 'lifebr', 'againbr', 'outbr', 'theyd', 'therebr', 'reallife', 'writerdirector', 'betterbr', 'moviesbr', 'youbr', 'watchbr', 'badbr', 'filmsbr', 'toobr', 'herbr', 'theyll', 'upbr', 'oneliners', 'hitchcocks', 'spoilersbr', 'charactersbr', 'itll', 'mustsee', 'scenebr', 'peoplebr', 'screenbr', 'workbr', 'bebr', 'morebr', 'seenbr', 'showbr', 'everyones', 'funnybr', 'worldbr', 'etcbr', 'timesbr', 'inbr', 'madefortv', 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.532
Accuracy Score:  0.53224
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
Converted 26818 words (3181 misses)
Missed words:  ['itbr', 'moviebr', 'filmbr', 'youve', 'werent', 'timebr', 'shouldnt', 'thembr', 'hadnt', 'lowbudget', 'himbr', 'theyve', 'onebr', 'thatbr', 'thisbr', 'mebr', 'storybr', 'bmovie', 'herebr', 'wellbr', 'allbr', 'goodbr', 'couldve', 'overthetop', 'onbr', 'wouldve', 'endbr', 'isbr', 'waybr', 'lifebr', 'againbr', 'outbr', 'theyd', 'therebr', 'reallife', 'writerdirector', 'betterbr', 'moviesbr', 'youbr', 'watchbr', 'badbr', 'filmsbr', 'toobr', 'herbr', 'theyll', 'upbr', 'oneliners', 'hitchcocks', 'spoilersbr', 'charactersbr', 'itll', 'mustsee', 'scenebr', 'peoplebr', 'screenbr', 'workbr', 'bebr', 'morebr', 'seenbr', 'showbr', 'everyones',

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.539
Accuracy Score:  0.53864
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
Converted 26818 words (3181 misses)
Missed words:  ['itbr', 'moviebr', 'filmbr', 'youve', 'werent', 'timebr', 'shouldnt', 'thembr', 'hadnt', 'lowbudget', 'himbr', 'theyve', 'onebr', 'thatbr', 'thisbr', 'mebr', 'storybr', 'bmovie', 'herebr', 'wellbr', 'allbr', 'goodbr', 'couldve', 'overthetop', 'onbr', 'wouldve', 'endbr', 'isbr', 'waybr', 'lifebr', 'againbr', 'outbr', 'theyd', 'therebr', 'reallife', 'writerdirector', 'betterbr', 'moviesbr', 'youbr', 'watchbr', 'badbr', 'filmsbr', 'toobr', 'herbr', 'theyll', 'upbr', 'oneliners', 'hitchcocks', 'spoilersbr', 'charactersbr', 'itll', 'mustsee', 'scenebr', 'peoplebr', 'screenbr', 'workbr', 'bebr', 'morebr', 'seenbr', 'showbr', 'everyones', 'funnybr', 'worldbr', 'etcbr', 'timesbr', 'inbr', 'madefortv', 'thought

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.507
Accuracy Score:  0.50744
----------

---------------------------------Threshold =  0.6 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
Converted 26818 words (3181 misses)
Missed words:  ['itbr', 'moviebr', 'filmbr', 'youve', 'werent', 'timebr', 'shouldnt', 'thembr', 'hadnt', 'lowbudget', 'himbr', 'theyve', 'onebr', 'thatbr', 'thisbr', 'mebr', 'storybr', 'bmovie', 'herebr', 'wellbr', 'allbr', 'goodbr', 'couldve', 'overthetop', 'onbr', 'wouldve', 'endbr', 'isbr', 'waybr', 'lifebr', 'againbr', 'outbr', 'theyd', 'therebr', 'reallife', 'writerdirector', 'betterbr', 'moviesbr', 'youbr', 'watchbr', 'badbr', 'filmsbr', 'toobr', 'herbr', 'theyll', 'upbr', 'oneliners', 'hitchcocks', 'spoilersbr', 'charactersbr', 'itll', 'mustsee', 'scenebr', 'peoplebr', 'screenbr', 'workbr', 'bebr', 'morebr', 'seenbr', 'showbr', 'everyones', 'funnybr', 'worldbr', 'etcbr', 'timesbr', 'inbr', 'madefortv', 'thought

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


KeyboardInterrupt: ignored

In [ ]:
df_mlp_imdb_2k.to_excel('tf-mlp_imdb_GloVE.xlsx')

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

--2021-06-29 06:10:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-29 06:10:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-29 06:10:05--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

hit_words = []
missed_words = []

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
        hit_words.append(word)
    else:
        misses += 1
        missed_words.append(word)
print("Converted %d words (%d misses)" % (hits, misses))

print("Missed words: ", missed_words)

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

# NewsGroup


1.   MLP



In [ ]:
import os

import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

data = fetch_20newsgroups(subset='train', categories=None)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()


from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    # print('y_train:', y_train)
    with tf.device('/device:GPU:0'):
      clf.fit(X_train, y_train)
      y_pred = np.array(clf.predict(X_test))
      
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)
    # dict1['roc-auc'] = roc_auc_score(y_test, y_pred, multi_class="ovr")
    dict1['precision'] = precision_score(y_test, y_pred, average='macro')
    dict1['recall'] = recall_score(y_test, y_pred, average='macro')
    dict1['f1'] = f1_score(y_test, y_pred, average='macro')

    print("-" * 10)
    print()

    return dict1

11314 documents
20 categories



In [ ]:
#------------------For GloVE---------------------#
embedding_dim = 300
maxlen = 500


def get_embed_matrix(word_index):

    hits = 0
    misses = 0

    hit_words = []
    missed_words = []
    # Prepare embedding matrix
    

    embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (max_words, 100)
    for word, i in word_index.items():
        if i < max_words:
            embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros
                hits += 1
                hit_words.append(word)
            else:
                misses += 1
                missed_words.append(word)  


                
    print("Converted %d words (%d misses)" % (hits, misses))

    print("Missed words: ", missed_words)

    return embedding_matrix


    #------------------For GloVE---------------------#

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

tf.random.set_seed(0)
def create_model(word_index):

    # int_sequences_input = keras.Input(shape=(None,), dtype="int64")
    # embedded_sequences = embedding_layer(int_sequences_input)
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_words, embedding_dim, input_length = maxlen),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(50),
    ])

    embedding_matrix = get_embed_matrix(word_index)
    model.layers[0].set_weights([embedding_matrix])
    model.layers[0].trainable = False

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  # loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
# def vectorize_data(data, vocab: dict) -> list:
#     keys = list(vocab.keys())
#     filter_unknown = lambda word: vocab.get(word, None) is not None
#     encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
#     vectorized = list(map(encode, data))
#     print('Vectorize sentences... (done)')
#     return vectorized

In [ ]:
%%time
# MLP for NG
from sklearn.neural_network import MLPClassifier

# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases

from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

# Parameters
# mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=100,activation = 'relu',solver='adam',random_state=1,learning_rate_init=0.01,
#                   learning_rate='adaptive')
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8, max_features=30000)


df_mlp_ng = pd.DataFrame()

n_list = [10, 20, 30, 40, 50]
kbest_list=[4, 5, 6, 7, 8]
threshold = [0.4, 0.5, 0.6, 0.7, 0.8]


X, y = data.data, data.target
# embedding_vector_size = 256
# trigrams_model = Word2Vec(sentences = X, size = embedding_vector_size, min_count=3, window=5, workers=4)

CPU times: user 7.35 ms, sys: 14.1 ms, total: 21.5 ms
Wall time: 21.4 ms


In [ ]:
%%time
path_to_glove_file =  "glove.6B."+str(embedding_dim)+"d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.
CPU times: user 16.4 s, sys: 849 ms, total: 17.2 s
Wall time: 17.2 s


In [ ]:
# MLP for NG
# Parameters
# mnb_params = 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing. sequence import pad_sequences



mlp_params = dict(hidden_layer_sizes=(100,50), max_iter=50,activation = 'relu',solver='adam',random_state=1)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8, max_features=30000)

for n in n_list:    
  tf.keras.backend.clear_session()  
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

  # print("Supervised SGDClassifier on 100% of the data:")
  # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

  np.random.seed(0)
  # print("Shape before: ", len(X_train), len(X_test))

  X_train = np.array([i for i in X_train])
  X_test = np.array([i for i in X_test])

  # print("Shape middle: ", X_train.shape, X_test.shape)
  # print(X_train[0])

  tokenizer = Tokenizer(num_words=max_words)
  tokenizer.fit_on_texts(np.concatenate((X_train, X_test)))
  sequences = tokenizer.texts_to_sequences(np.concatenate((X_train, X_test)))
  word_index = tokenizer.word_index                   
  print("Found %s unique tokens." % len(word_index)) 

  data = pad_sequences(sequences, maxlen=maxlen)

  X_train, X_test = data[:len(X_train)], data[len(X_train):]

  # print("Shape after: ", X_train.shape, X_test.shape)
  # print(X_train[0])



  unlabeled_mask = np.random.rand(len(y_train)) < 0.5
  X_u50, y_u50 = map(list, zip(*((x, y)
                  for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
  
  X_u50 = np.array(X_u50)
  y_u50 = np.array([-1 for i in y_u50])

  X_50, y_50 = map(list, zip(*((x, y)
            for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
  X_50 = np.array(X_50)
  y_50 = np.array(y_50)





  percentage = 2*(n/100)
  y_mask = np.random.rand(len(y_50)) < percentage

  # X_20 and y_20 are the subset of the train dataset indicated by the mask
  X_20, y_20 = map(list, zip(*((x, y)
                  for x, y, m in zip(X_50, y_50, y_mask) if m)))
  X_20 = np.array(X_20)
  y_20 = np.array(y_20)

  print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

  # print("Shape after after: ", X_20.shape)
  # break




  # print(len(X_20),"SHAPE")


  model = KerasClassifier(build_fn=lambda: create_model(word_index), epochs=20, verbose=0)
  pipeline = Pipeline([
      # ('vect', CountVectorizer(**vectorizer_params)),
      # ('tfidf', TfidfTransformer()), 
      # # ('kbest', MySelectKBest(f_classif, k=20000)),         
      # ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
      ('clf', model)
  ])

  



  temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
  df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)

  # set the non-masked subset to be unlabeled
  # set only 50% of data to be unlabeled in every iteration of training.
  print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
        "is unlabeled):")
  for t in threshold:
    print("---------------------------------Threshold = ", t,"---------------------------------")
    tf.keras.backend.clear_session()
  
  # X_50, y_50 = map(list, zip(*((x, y)
  #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
    # SelfTraining Pipeline
    model = KerasClassifier(build_fn=lambda: create_model(word_index), epochs=20, verbose=0)
    st_pipeline = Pipeline([
        # ('vect', CountVectorizer(**vectorizer_params)),
        # ('tfidf', TfidfTransformer()),
        # # ('kbest', MySelectKBest(f_classif, k=20000)),
        # ('dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
        ('clf', SelfTrainingClassifier(model, criterion = 'threshold', threshold = t, verbose=True)),
    ])

    


    temp = eval_and_print_metrics_df(st_pipeline, np.concatenate((X_20,X_u50)), np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
    df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)




~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Found 134142 unique tokens.
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.233
Accuracy Score:  0.23294450335807707
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 4307 new labels.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.192
Accuracy Score:  0.1919406150583245
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 4307 new labels.


...

Micro-averaged F1 score on test set: 0.458
Accuracy Score:  0.45846588900671614
----------

---------------------------------Threshold =  0.7 ---------------------------------
Number of training samples: 8485
Unlabeled samples in training set: 4307
End of iteration 1, added 4307 new labels.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.454
Accuracy Score:  0.45422410745846586
----------

---------------------------------Threshold =  0.8 ---------------------------------
Number of training samples: 8485
Unlabeled samples in training set: 4307
End of iteration 1, added 4307 new labels.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Micro-averaged F1 score on test set: 0.445
Accuracy Score:  0.4446800989749028
----------



In [ ]:
df_mlp_ng.to_excel('tf-mlp_ng_GloVE.xlsx')

# Enron

1. MLP